In [1]:
pip install transformers torch

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

model_name = "shiprocket-ai/open-tinybert-indian-address-ner"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/266M [00:00<?, ?B/s]

Device set to use mps:0


In [19]:
text = "Flat 302, Tower B, Prestige Shantiniketan, Whitefield, Bengaluru, Karnataka 560066"

results = ner_pipeline(text)
for r in results:
    print(f"{r['word']:<25} | {r['entity_group']:<15} | {r['score']:.3f}")

flat 302,                 | house_details   | 0.992
tower b, prestige shantiniketan, | building_name   | 0.928
whitefield,               | locality        | 0.975
bengaluru,                | city            | 0.987
karnataka                 | state           | 0.998
560066                    | pincode         | 1.000


In [ ]:
addresses = [
    "Plot 12, Sector 4, Dwarka, New Delhi, Delhi 110075",
    "No. 5, MG Road, Camp, Pune, Maharashtra 411001"
]

for addr in addresses:
    print("\nAddress:", addr)
    for ent in ner_pipeline(addr):
        print(ent["word"], "->", ent["entity_group"])



Address: Plot 12, Sector 4, Dwarka, New Delhi 110075
plot 12, -> house_details
sector -> locality
4 -> road
, dwarka, -> locality
new delhi -> city
110075 -> pincode

Address: No. 5, MG Road, Camp, Pune 411001
no. 5, -> house_details
mg road, -> road
camp, -> landmarks
pune -> city
411001 -> pincode


In [9]:
import json
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from sklearn.metrics import classification_report

# --- Load dataset ---
with open("ner_list_test.json", "r") as f:
    data = json.load(f)

# --- Load model + tokenizer ---
model_name = "shiprocket-ai/open-tinybert-indian-address-ner"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

# --- Label mapping (align model labels to your schema) ---
label_map = {
    "BUILDING_NAME": "UNIT",
    "HOUSE_NUMBER": "UNIT",
    "HOUSE_DETAILS": "UNIT",
    "AREA": "AREA_NAME",
    "LOCALITY": "AREA_NAME",
    "ROAD": "STREET_NAME",
    "STREET": "STREET_NAME",
    "PINCODE": "AREA_PINCODE",
    "LANDMARKS": "LANDMARK",
    "LANDMARK": "LANDMARK",
    "CITY": "CITY_NAME",
    "STATE": "STATE_NAME"
}

# --- Initialize collectors ---
true_pairs, pred_pairs = [], []

# --- Iterate through each example ---
for entry in data:
    text = entry["text"]
    # ground truth: (entity_text, label)
    gt = {(ent[2].strip().lower(), ent[3].upper()) for ent in entry["entities"]}
    
    # model predictions
    preds = ner_pipeline(text)
    pred_set = set()
    for p in preds:
        ent_text = p["word"].strip().lower()
        ent_label = label_map.get(p["entity_group"].upper(), p["entity_group"].upper())
        pred_set.add((ent_text, ent_label))
    
    true_pairs.extend(list(gt))
    pred_pairs.extend(list(pred_set))

# --- Extract labels for span-level comparison ---
y_true, y_pred = [], []

# unique labels in your dataset + predictions
unique_labels = sorted(list({lbl for _, lbl in true_pairs + pred_pairs}))

for lbl in unique_labels:
    for pair in set(true_pairs + pred_pairs):
        is_true = pair in true_pairs and pair[1] == lbl
        is_pred = pair in pred_pairs and pair[1] == lbl
        y_true.append(is_true)
        y_pred.append(is_pred)

# --- Generate report ---
print("\n=== Span-Level Evaluation (Text + Label Match) ===\n")
print(classification_report(y_true, y_pred, digits=3))

Device set to use mps:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



=== Span-Level Evaluation (Text + Label Match) ===

              precision    recall  f1-score   support

       False      0.955     0.960     0.957    121818
        True      0.213     0.191     0.201      6871

    accuracy                          0.919    128689
   macro avg      0.584     0.576     0.579    128689
weighted avg      0.915     0.919     0.917    128689



In [8]:
import json
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from sklearn.metrics import classification_report
from fuzzywuzzy import fuzz

# --- Load dataset ---
with open("ner_list_test.json", "r") as f:
    data = json.load(f)

# --- Load model + tokenizer ---
model_name = "shiprocket-ai/open-tinybert-indian-address-ner"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

# --- Label mapping (TinyBERT → Your Schema) ---
label_map = {
    "BUILDING_NAME": "UNIT",
    "HOUSE_DETAILS": "UNIT",
    "FLOOR": "UNIT",
    "ROAD": "STREET_NAME",
    "LOCALITY": "AREA_NAME",
    "SUB_LOCALITY": "AREA_NAME",
    "CITY": "CITY_NAME",
    "STATE": "STATE_NAME",
    "PINCODE": "AREA_PINCODE",
    "LANDMARKS": "LANDMARK",
    "COUNTRY": "UNASSIGNED"
}

# --- Initialize collectors ---
true_pairs, pred_pairs = [], []

# --- Iterate through each example ---
for entry in data:
    text = entry["text"]
    
    # ground truth (text, label)
    gt = {(ent[2].strip().lower(), ent[3].upper()) for ent in entry["entities"]}
    
    # model predictions
    preds = ner_pipeline(text)
    pred_set = set()
    for p in preds:
        ent_text = p["word"].strip().lower()
        ent_label = label_map.get(p["entity_group"].upper(), p["entity_group"].upper())
        pred_set.add((ent_text, ent_label))
    
    true_pairs.extend(list(gt))
    pred_pairs.extend(list(pred_set))

# --- Fuzzy Matching Function ---
def fuzzy_match(pair_true, pair_pred, threshold=85):
    """Return True if labels match and text similarity >= threshold."""
    text_true, label_true = pair_true
    text_pred, label_pred = pair_pred
    if label_true != label_pred:
        return False
    similarity = fuzz.ratio(text_true, text_pred)
    return similarity >= threshold

# --- Compute fuzzy-matched pairs ---
matched_pairs = set()
for t_pair in true_pairs:
    for p_pair in pred_pairs:
        if fuzzy_match(t_pair, p_pair):
            matched_pairs.add((t_pair, p_pair))
            break  # avoid double counting

# --- Convert to binary indicators for sklearn ---
y_true, y_pred = [], []
unique_labels = sorted(list({lbl for _, lbl in true_pairs + pred_pairs}))

for lbl in unique_labels:
    # get label-specific sets
    true_lbl_pairs = {t for t in true_pairs if t[1] == lbl}
    pred_lbl_pairs = {p for p in pred_pairs if p[1] == lbl}
    
    for pair in true_lbl_pairs | pred_lbl_pairs:
        is_true = pair in true_lbl_pairs
        # check if any pred matched via fuzzy logic
        is_pred = (
            pair in pred_lbl_pairs
            or any(fuzzy_match(pair, p) for p in pred_lbl_pairs)
        )
        y_true.append(is_true)
        y_pred.append(is_pred)

# --- Generate report ---
print("\n=== Fuzzy Span-Level Evaluation (Text + Label Match, ≥85% Similarity) ===\n")
print(classification_report(y_true, y_pred, digits=3))

/Users/sarang/Library/Python/3.13/lib/python/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
Device set to use mps:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



=== Fuzzy Span-Level Evaluation (Text + Label Match, ≥85% Similarity) ===

              precision    recall  f1-score   support

       False      0.000     0.000     0.000      4759
        True      0.317     0.321     0.319      6871

    accuracy                          0.190     11630
   macro avg      0.159     0.161     0.160     11630
weighted avg      0.187     0.190     0.189     11630



In [15]:
import json
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from collections import defaultdict
import re

# --- Load dataset ---
with open("ner_list_test.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# --- Load model + tokenizer ---
model_name = "shiprocket-ai/open-tinybert-indian-address-ner"  # or your fine-tuned one
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

# --- Label mapping ---
label_map = {
    "BUILDING_NAME": "UNIT",
    "HOUSE_NUMBER": "UNIT",
    "HOUSE_DETAILS": "UNIT",
    "FLOOR": "UNIT",
    "SUB_LOCALITY": "UNIT",
    "AREA": "AREA_NAME",
    "LOCALITY": "AREA_NAME",
    "ROAD": "STREET_NAME",
    "STREET": "STREET_NAME",
    "PINCODE": "AREA_PINCODE",
    "LANDMARKS": "LANDMARK",
    "LANDMARK": "LANDMARK",
    "CITY": "CITY_NAME",
    "STATE": "STATE_NAME"
}

# --- Helper: normalize ---
def normalize_text(text):
    return re.sub(r"[^a-z0-9 ]+", " ", text.lower()).strip()

# --- Helper: fuzzy match ---
def fuzzy_match(span1, span2):
    text1, label1 = span1
    text2, label2 = span2
    if label1 != label2:
        return False
    n1, n2 = normalize_text(text1), normalize_text(text2)
    # loose partial / token overlap match
    return (
        n1 in n2
        or n2 in n1
        or len(set(n1.split()) & set(n2.split())) / max(len(n1.split()), len(n2.split()), 1) > 0.90
    )

# --- Collect pairs ---
true_pairs, pred_pairs = [], []

for entry in data:
    text = entry["text"]
    
    # ground truth
    gt = {(ent[2].strip().lower(), ent[3].upper()) for ent in entry["entities"]}
    
    # predictions
    preds = ner_pipeline(text)
    pred_set = set()
    for p in preds:
        ent_text = p["word"].strip().lower()
        ent_label = label_map.get(p["entity_group"].upper(), p["entity_group"].upper())
        pred_set.add((ent_text, ent_label))
    
    true_pairs.extend(list(gt))
    pred_pairs.extend(list(pred_set))

# --- Compute overall fuzzy metrics ---
tp = sum(1 for t in true_pairs if any(fuzzy_match(t, p) for p in pred_pairs))
fp = sum(1 for p in pred_pairs if not any(fuzzy_match(p, t) for t in true_pairs))
fn = sum(1 for t in true_pairs if not any(fuzzy_match(t, p) for p in pred_pairs))

precision = tp / (tp + fp + 1e-6)
recall = tp / (tp + fn + 1e-6)
f1 = 2 * precision * recall / (precision + recall + 1e-6)

print("\n=== Fuzzy Span-Level Evaluation (Text + Label Match, ≥85% Similarity) ===")
print(f"Precision: {precision:.3f}")
print(f"Recall:    {recall:.3f}")
print(f"F1-score:  {f1:.3f}")
print(f"(TP={tp}, FP={fp}, FN={fn})")

# --- Per-label metrics ---
label_stats = defaultdict(lambda: {"tp": 0, "fp": 0, "fn": 0})

for t_pair in true_pairs:
    label = t_pair[1]
    if any(fuzzy_match(t_pair, p) for p in pred_pairs):
        label_stats[label]["tp"] += 1
    else:
        label_stats[label]["fn"] += 1

for p_pair in pred_pairs:
    label = p_pair[1]
    if not any(fuzzy_match(p_pair, t) for t in true_pairs):
        label_stats[label]["fp"] += 1

print("\n=== Per-Entity Fuzzy F1 Breakdown ===\n")
for lbl, s in label_stats.items():
    tp, fp, fn = s["tp"], s["fp"], s["fn"]
    prec = tp / (tp + fp + 1e-6)
    rec = tp / (tp + fn + 1e-6)
    f1 = 2 * prec * rec / (prec + rec + 1e-6)
    print(f"{lbl:<15}  P={prec:.3f}  R={rec:.3f}  F1={f1:.3f}  (TP={tp}, FP={fp}, FN={fn})")

Device set to use mps:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



=== Fuzzy Span-Level Evaluation (Text + Label Match, ≥85% Similarity) ===
Precision: 0.989
Recall:    0.923
F1-score:  0.955
(TP=6998, FP=81, FN=584)

=== Per-Entity Fuzzy F1 Breakdown ===

STREET_NAME      P=0.998  R=1.000  F1=0.999  (TP=847, FP=2, FN=0)
AREA_NAME        P=0.998  R=1.000  F1=0.999  (TP=2207, FP=5, FN=0)
UNIT             P=1.000  R=1.000  F1=1.000  (TP=2520, FP=0, FN=0)
SOCIETY_NAME     P=0.000  R=0.000  F1=0.000  (TP=0, FP=0, FN=490)
LANDMARK         P=0.920  R=1.000  F1=0.958  (TP=677, FP=59, FN=0)
CITY_NAME        P=0.984  R=1.000  F1=0.992  (TP=656, FP=11, FN=0)
AREA_PINCODE     P=0.963  R=0.912  F1=0.937  (TP=52, FP=2, FN=5)
STATE_NAME       P=0.951  R=0.661  F1=0.780  (TP=39, FP=2, FN=20)
UNASSIGNED       P=0.000  R=0.000  F1=0.000  (TP=0, FP=0, FN=69)


In [16]:
for i, entry in enumerate(data):
    print(f"\n🔹 Address {i+1}: {entry['text']}")
    print("Ground Truth:", [e[3] for e in entry["entities"]])
    print("Predicted   :", [p["entity_group"] for p in ner_pipeline(entry["text"])])



🔹 Address 1: sudarshnam t.c 5/29142 parayankonam lane, cheruvikkal sreekaryam
Ground Truth: ['unit', 'unit', 'street_name', 'area_name']
Predicted   : ['house_details', 'road', 'landmarks', 'locality', 'landmarks', 'landmarks']

🔹 Address 2: f-1-260 mahavir enclave gali no 3 near family store
Ground Truth: ['unit', 'street_name', 'society_name', 'landmark']
Predicted   : ['house_details', 'building_name', 'road', 'landmarks']

🔹 Address 3: house no 23 ward 06 nawarangpur patsendri old route no nh 216, saraipali subdistrict
Ground Truth: ['unit', 'street_name', 'society_name', 'area_name', 'area_name', 'city_name']
Predicted   : ['house_details', 'locality', 'road', 'locality']

🔹 Address 4: 703, 10th cross hasa road , banglore
Ground Truth: ['unit', 'street_name', 'city_name']
Predicted   : ['house_details', 'road', 'locality']

🔹 Address 5: 19 , mahesh tenament. nr hirwadi road ahmedabad
Ground Truth: ['unit', 'unit', 'city_name', 'landmark']
Predicted   : ['building_name', 'road', '

KeyboardInterrupt: 

In [6]:
from transformers import pipeline

ner_pipeline = pipeline(
    "ner",
    model="shiprocket-ai/open-tinybert-indian-address-ner",
    aggregation_strategy="simple"
)

for i, entry in enumerate(data):
    print(f"\n🔹 Address {i+1}: {entry['text']}")
    print("Ground Truth:", [e[3] for e in entry["entities"]])  
    preds = ner_pipeline(entry["text"])
    print("Predicted   :", [p["entity_group"] for p in preds])


Device set to use mps:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



🔹 Address 1: sudarshnam t.c 5/29142 parayankonam lane, cheruvikkal sreekaryam
Ground Truth: ['unit', 'unit', 'street_name', 'area_name']
Predicted   : ['house_details', 'road', 'landmarks', 'locality', 'landmarks', 'landmarks']

🔹 Address 2: f-1-260 mahavir enclave gali no 3 near family store
Ground Truth: ['unit', 'street_name', 'society_name', 'landmark']
Predicted   : ['house_details', 'building_name', 'road', 'landmarks']

🔹 Address 3: house no 23 ward 06 nawarangpur patsendri old route no nh 216, saraipali subdistrict
Ground Truth: ['unit', 'street_name', 'society_name', 'area_name', 'area_name', 'city_name']
Predicted   : ['house_details', 'locality', 'road', 'locality']

🔹 Address 4: 703, 10th cross hasa road , banglore
Ground Truth: ['unit', 'street_name', 'city_name']
Predicted   : ['house_details', 'road', 'locality']

🔹 Address 5: 19 , mahesh tenament. nr hirwadi road ahmedabad
Ground Truth: ['unit', 'unit', 'city_name', 'landmark']
Predicted   : ['building_name', 'road', '